# preprocessing data

In [15]:

#!/usr/bin/env python
#-*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import time
import requests  # pip install requests
import json
import re
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
import xml.etree.ElementTree as ET


#remove accents vietnamese
def remove_accents(text):
    s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
    s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
#     s1.encode('utf-8')
#     s0.encode('utf-8')
    if text == np.nan or text == np.NAN or text == np.NaN:
        return text
    else:    
        s = ''
        text = str(text)
        text = text.decode('utf-8')

        for c in text:
            if c in s1:
                s += s0[s1.index(c)]
            else:
                s += c
        return s.strip().replace(" ","_").replace(",", "-")
     

In [16]:
#connect to our cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': '172.16.3.105', 'port': 9200}])
# res = requests.get('http://172.16.3.105:9200/index_product/product/86734')
# print(res.content)

# get list category ID
def get_list_cate_ID(list_product_ID):  
    global list_cate_ID
    list_cate_ID = []
#     f = open('product_cate.txt', 'wb')
    for productID in list_product_ID:
        try:
            pathCate = es.get(index="index_product", doc_type="product", id=str(productID))['_source']['categoryPath']
            listCate = pathCate.split(" ")
#             f.write(str(productID) +','+listCate[-1]+"\n")
            list_cate_ID.append(listCate[-1])
        except Exception as error:
            print(error)
#     f.close()
    return list_cate_ID


def get_cate_ID_by_proID(productID):
    try:
        pathCate = es.get(index="index_product", doc_type="product", id=str(productID))['_source']['categoryPath']
        listCate = pathCate.split(" ")
        return int(listCate[-1])
    except Exception as error:
        return np.nan
    

def get_name_cate_by_cate_ID(cate_ID):
    try: 
        url = 'http://webservice.vuivui.com/productsvc.asmx/GetProductCategoryByCate?intSiteID=8&intCategoryID='
        url = url + str(int(cate_ID))
        res = requests.get(url)
        xml = ET.fromstring(res.content)
        return xml[0][1].text
    except:
        return ""
def get_list_name_cate(list_cate_ID, filename = 'cate_name.txt'):
    f = open(filename, 'wb')
    for cate_ID in list_cate_ID:
        f.write(cate_ID + ',' + get_name_cate_by_cate_ID(cate_ID) + '\n')
    f.close()    

In [3]:

start = time.time()
data_file = pd.read_csv('/home/tgdd/TA_69/vuivui/20170518161748.csv')
data1 = data_file[['SaleOrderID','ProductID','Quantity']]
# data1['CateID'] = data1["ProductID"].apply(get_cate_ID_by_proID)
# data1['CateID_'] = data1['CateID']
# data1['ProductID_']=data1['ProductID']
# data1['NameCate'] = data1['CateID'].apply(get_name_cate_by_cate_ID).apply(remove_accents)

list_product_ID = list(set(data1.ProductID))
df = pd.DataFrame(list_product_ID, columns=['ProductID'])
df['CateID'] = df.ProductID.apply(get_cate_ID_by_proID)
df['CateID_'] = df['CateID']
df = df.dropna(how = 'any', subset=['CateID'])
df = df[~df.CateID.isin(['3103', '2102'])]
# df['NameCate']= df.CateID.apply(get_name_cate_by_cate_ID)
# df['NameCate_']=df['NameCate'].apply(remove_accents)
data1 = data1.merge(df, how='left', on = 'ProductID')
# data1['NameCate_2']=data1['NameCate_']
data1.to_csv('/home/tgdd/TA_69/vuivui/dataset_SaleOrder.csv', index = False)
time.time() - start

18.655596017837524

In [ ]:
# don't need to run above command

In [17]:
data1 = pd.read_csv('/home/tgdd/TA_69/vuivui/dataset_SaleOrder.csv')


In [4]:
# data2.to_csv('/home/tgdd/TA_69/vuivui/data_vuivui.csv',encoding='utf-8')

# ew = pd.ExcelWriter('/home/tgdd/TA_69/vuivui/test.xlsx', options={'encoding':'utf-8'})
# data2.to_excel(ew, 'test')
# ew.save()

In [18]:
#
# data2 = data1.pivot_table('NameCate_2', index = 'SaleOrderID',columns='NameCate_',aggfunc=lambda x: ' '.join(str(v) for v in x))
data2 = data1.pivot_table('CateID', index = 'SaleOrderID',columns='CateID_',aggfunc=lambda x: ' '.join(str(v) for v in x))
data2.to_csv('big_df.txt',header=False, index=False)#write dataframe to text file, separate = ","
f1 = open('sparse_df.txt', 'wb')#read file big_df.txt then write into sparse_df.txt, sparse dataframe
f = open('big_df.txt', 'rb')
for line in f:
    line = ' '.join(line.split(','))
    a = ','.join(line.split())
    if(len(a)>0):
        f1.write(a+'\n')
f1.close()
f.close()


In [6]:
# # data2.columns = data2.columns.droplevel(0) #remove amount
# data2.columns.name = None               #remove categories
# data2 = data2.reset_index() 

# start = time.time()
# # write dataframe to text file, it not good due to run slowlly (shouldn't use)
# f = open("df_to_text.txt", 'wb')
# for i in range(data2.shape[0]):
#     for j in range(data2.shape[1]):
#         if(data2.iloc[i][j] > 0):
#             f.write(str(data2.columns[j]) + ',')
#     f.write('\n')
# f.close()

# duration = time.time() - start            

# Apriori algorithm

In [19]:
"""
Usage:
    $python apriori.py -f DATASET.csv -s minSupport  -c minConfidence
    $python apriori.py -f DATASET.csv -s 0.15 -c 0.6
"""

import sys
import time
from itertools import chain, combinations
from collections import defaultdict
from optparse import OptionParser

def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])

def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
        """calculates the support for items in the itemSet and returns a subset
       of the itemSet each of whose elements satisfies the minimum support"""
        _itemSet = set()
        localSet = defaultdict(int)

        for item in itemSet:
                for transaction in transactionList:
                        if item.issubset(transaction):
                                freqSet[item] += 1
                                localSet[item] += 1

        for item, count in localSet.items():
                support = float(count)/len(transactionList)

                if support >= minSupport:
                        _itemSet.add(item)

        return _itemSet


def joinSet(itemSet, length):
        """Join a set with itself and returns the n-element itemsets"""
        return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


def getItemSetTransactionList(data_iterator):
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))              # Generate 1-itemSets
    return itemSet, transactionList


def runApriori(data_iter, minSupport, minConfidence):
    """
    run the apriori algorithm. data_iter is a record iterator
    Return both:
     - items (tuple, support)
     - rules ((pretuple, posttuple), confidence)
    """
    itemSet, transactionList = getItemSetTransactionList(data_iter)

    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key=n-itemSets,value=support)
    # which satisfy minSupport

    assocRules = dict()
    # Dictionary which stores Association Rules

    oneCSet = returnItemsWithMinSupport(itemSet,
                                        transactionList,
                                        minSupport,
                                        freqSet)

    currentLSet = oneCSet
    k = 2
    while(currentLSet != set([])):
        largeSet[k-1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(currentLSet,
                                                transactionList,
                                                minSupport,
                                                freqSet)
        currentLSet = currentCSet
        k = k + 1

    def getSupport(item):
            """local function which Returns the support of an item"""
            return float(freqSet[item])/len(transactionList)

    toRetItems = []
    for key, value in largeSet.items():
        toRetItems.extend([(tuple(item), getSupport(item))
                           for item in value])

    toRetRules = []
    for key, value in largeSet.items()[1:]:
        for item in value:
            _subsets = map(frozenset, [x for x in subsets(item)])
            for element in _subsets:
                remain = item.difference(element)
                if len(remain) > 0:
                    confidence = getSupport(item)/getSupport(element)
                    if confidence >= minConfidence:
                        toRetRules.append(((tuple(element), tuple(remain)),
                                           confidence))
    return toRetItems, toRetRules


def printResults(items, rules):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    for item, support in sorted(items, key=lambda (item, support): support):
        print "item: %s , %.3f" % (str(item), support)
    print "\n------------------------ RULES:"
    for rule, confidence in sorted(rules, key=lambda (rule, confidence): confidence):
        pre, post = rule
        print "Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence)
def saveResults(items, rules):
    """save the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    f = open('results.txt', 'wb')
    for item, support in sorted(items, key=lambda (item, support): support):
        f.write("\nitem: %s , %.3f" % (str(item), support)) 
    f.write("\n------------------------ RULES:")
    i = 0
    for rule, confidence in sorted(rules, key=lambda (rule, confidence): confidence):
        pre, post = rule
        i+=1
        f.write("\nRule %s : %s ==> %s , %.3f" % (str(i), str(pre), str(post), confidence)) 

def dataFromFile(fname):
        """Function which reads from the file and yields a generator"""
        file_iter = open(fname, 'rU')
        for line in file_iter:
                line = line.strip().rstrip(',')                         # Remove trailing comma
                record = frozenset(line.split(','))
                yield record


if __name__ == "__main__":
#     optparser = OptionParser()
#     optparser.add_option('-f', '--inputFile',
#                          dest='input',
#                          help='filename containing csv',
#                          default=None)
#     optparser.add_option('-s', '--minSupport',
#                          dest='minS',
#                          help='minimum support value',
#                          default=0.15,
#                          type='float')
#     optparser.add_option('-c', '--minConfidence',
#                          dest='minC',
#                          help='minimum confidence value',
#                          default=0.6,
#                          type='float')

#     (options, args) = optparser.parse_args()

#     inFile = None
#     if options.input is None:
#             inFile = sys.stdin
#     elif options.input is not None:
#             inFile = dataFromFile(options.input)
#     else:
#             print 'No dataset filename specified, system with exit\n'
#             sys.exit('System will exit')

#     minSupport = options.minS
#     minConfidence = options.minC

    start = time.time()
    inFile  = dataFromFile('sparse_df.txt')
    minSupport, minConfidence = 0.0005, 0.5
    
    items, rules = runApriori(inFile, minSupport, minConfidence)
#     printResults(items, rules)
    saveResults(items, rules)
    dur =  str(time.time() - start)

In [8]:
dur

'98.1884000301'

# Web service


In [9]:
# import pymssql
from os import getenv
# -*- coding: utf-8 -*-
import tornado.ioloop
import tornado.web
import tornado.httpserver
import tornado.httpclient
import tornado.netutil
# import sys
from tornado.ioloop import IOLoop, PeriodicCallback
from threading import Thread
import time

class Test(tornado.web.RequestHandler):
#     def __init__(self):
#         self.a = HandClass(self)
#     def cong(self, a, b):
#         return a+b
    
    def get(self):
        self.write("Welcome to Thegioididong.com")

class TrackingViews(tornado.web.RequestHandler):

    def call(self):
        return cong(1, 2)
    
    def get(self):
        try:
            list_product = self.get_argument('list_Product', None)
#             producID = int(self.get_argument('ProductID', None))
#             num = int(self.get_argument('num', None))
#             self.write(str(list_product.split('_')))
            self.write(str(get_result(list_product, results)))
        except Exception as error:
            self.write("We have no recommend for " + list_product +" Error: "  + repr(error))

def cong(a, b):
    return a+b
   
def inital_():
    global results
    tmp = pd.DataFrame(rules)
    tmp1 = pd.DataFrame(list((tmp[0].values)))
    tmp1.columns = ['A', 'B']
    tmp2 = pd.pivot_table(tmp1, index='A', aggfunc=lambda x: ' '.join(str(v) for v in x))
    tmp2 = tmp2.reset_index()
    tmp2.B = tmp2['B'].apply(lambda row: [int(float(x)) for x in re.findall(r"\d+\.?\d*", str(row))])
    tmp2.A = tmp2['A'].apply(lambda row: list(int(float(x)) for x in row))
    results = tmp2

def get_result(list_product, results):#
    try:
        set_product = set(map(lambda x: int(x), list_product.split()))
        for index, row in results.iterrows():
            if set_product == set(row.A):
                recommend = row.B
                break        
    except Exception as error:
        return '' 
    return recommend    
 
# return all subset
def list_powerset(lst):
    return reduce(lambda result, x: result + [subset + [x] for subset in result], lst, [[]])

def get_results_powerset(list_product, results):   
    list_cate = list(set(map(lambda x: int(x), list_product.split())))
    recommend = []
    for item_set in list_powerset(list_cate):
        for index, row in results.iterrows():
            if set(item_set) == set(row.A):
                recommend.extend(row.B)
    return recommend    
    
    
class Application(tornado.web.Application):
    def __init__(self):
        handlers = [
                  (r"/recommendation", TrackingViews),
                  (r"/", Test),
                
        ]
        settings = {}
        tornado.web.Application.__init__(self, handlers, **settings)

def thread1(threadname):
    #global a       # Optional if you treat a as read-only
    while 1:
        http_server = tornado.httpserver.HTTPServer(Application())
        http_server.listen(8898)

def thread2(threadname):
    while 1:
        inital_()
        time.sleep(86400)           
    
if __name__ == "__main__":   
    results = 0

#     test = 0
    thread1 = Thread( target=thread1, args=("Thread-1", ) )
    thread2 = Thread( target=thread2, args=("Thread-2", ) )

#     thread3.start()
    thread1.start()
    time.sleep(1)
    thread2.start()   

In [ ]:
tornado.ioloop.IOLoop.instance().stop()

In [10]:
results.head()

,A,B
0,[2002],[2102]
1,"[2286, 2504]",[3103]
2,"[2289, 2286, 2803]","[2805, 2804]"
3,"[2289, 2387]",[2286]
4,"[2289, 2803]","[2286, 2804]"


In [9]:
list_product = '2289 2286 2803'

In [30]:

def get_results_powerset(list_product, results):   
    list_cate = list(set(map(lambda x: int(x), list_product.split())))
    recommend = []
    for item_set in list_powerset(list_cate):
        for index, row in results.iterrows():
            if set(item_set) == set(row.A):
                recommend.extend(row.B)
    return recommend 

In [20]:
start = time.time()
get_results_powerset(list_product)
time.time() -start

0.24366283416748047

In [35]:
start = time.time()
get_result(list_product, results)
time.time() -start

0.0008840560913085938

In [64]:
list_set = list_powerset(list_test)

In [13]:
list_set

NameError: name 'list_set' is not defined

In [26]:
len(data1.CateID.unique())


294

# Connect to Redis

In [11]:
# pip install redis-py-cluster
import redis
from rediscluster import StrictRedisCluster
# Requires at least one node for cluster discovery. Multiple nodes is recommended.
startup_nodes = [{"host": "10.1.12.30", "port": "16379"}]
rc = StrictRedisCluster(startup_nodes=startup_nodes, decode_responses=True)
#!redis-cli -h 10.1.12.30 -p 16379 -c
#rc.dbsize()

In [14]:
tmp2

NameError: name 'tmp2' is not defined

In [315]:
for index, row in tmp2.iterrows():
    rc.hset('RS',str(row.A), str(row.B))

In [12]:
rc.hget('RS', [2002])

In [13]:
rc.hgetall('RS')

{}

In [70]:
# sales = {'account': ['Jones LLC', 'Alpha Co', 'Blue Inc'],
#          'Jan': [150, 200, 50],
#          'Feb': [200, 210, 90],
#          'Mar': [140, 215, 95]}
# df = pd.DataFrame.from_dict(sales)
# df.reindex(df.account.str.len().sort_values().index)